In [58]:
import glob
import os

import dask.dataframe as dd
import pandas as pd
from dask.diagnostics import ProgressBar
from numpy.random import default_rng

In [59]:
maps = [
    "AutomatonAIE.SC2Map",
    "AbyssalReefAIE.SC2Map",
    "InterloperAIE.SC2Map",
    "EphemeronAIE.SC2Map",
    "AcropolisAIE.SC2Map",
    "ThunderbirdAIE.SC2Map",
]
test_maps = {maps[0]}

In [60]:
replay_dir = "../resources/replays/debug/*.SC2Replay.parquet"
replay_paths = glob.glob(replay_dir)
replay_names = [os.path.basename(p) for p in replay_paths]
replay_paths

['../resources/replays/aiarena\\3901860_Sharkling_PhantomBot_EphemeronAIE.SC2Replay.parquet']

In [61]:
def pipeline(replay_path: str) -> pd.DataFrame:
    replay_name = os.path.basename(replay_path)
    df = pd.read_parquet(replay_path)
    df = df.groupby(["game_loop", "unit_type", "player", "owner"])["tag"]
    df = df.count()
    df = df.rename("count")
    df = df.reset_index()
    df = df.assign(replay_name=replay_name)
    return df

In [62]:
with ProgressBar():
    df_all = dd.from_map(pipeline, replay_paths).compute()

[                                        ] | 0% Completed | 105.81 ms


OSError: Could not open Parquet input source '<Buffer>': Couldn't deserialize thrift: don't know what type: 


In [52]:
replay_names = df_all["replay_name"].unique()
test_replays = default_rng().choice(replay_names, len(replay_names) // 10, replace=False)
test_mask = df_all["replay_name"].isin(test_replays)
df_all[test_mask].reset_index().to_parquet("../resources/datasets/scout/test.parquet")
df_all[~test_mask].reset_index().to_parquet("../resources/datasets/scout/train.parquet")